In [50]:
%matplotlib inline

import glob
import pandas as pd
import yaml
import numpy as np
import cv2
import time

In [51]:
get_face_rect = False

In [52]:
path = '.'
prefix = '.\\'
wildcard = '\\*_'
persons = ["p{:03d}".format(i) for i in range(1,5)]
classes = [i for i in range(1,5)]
suffix = '_*_{:d}.bmp'
type = ['depth', 'ir1', 'ir2', 'rgb']

In [53]:
#p021 - class 3/4 errada

In [54]:
face_rect = [[120, 35],[120, 35], [80,35], [90, 30]]
face_window = 180

In [55]:
def get_face_rect_default(pclass, person, data, files_list):
    if len(files_list) >= 4:
        sample = [pclass, person]
        for i in range(4):
            sample.append(files_list[i])
            sample.append(face_rect[i][1])
            sample.append(face_rect[i][0])
            sample.append(face_rect[i][1]+face_window)
            sample.append(face_rect[i][0]+face_window)
        #print(sample)
        data.append(sample)

In [56]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def get_face_rectangle_viola_jones(pclass, person, data, files_list):
    images = []
    if len(files_list) >= 4:
        images = []
        for i in range(4):
            image = cv2.imread(files_list[i],0)
            #print(image.shape, face_rect[i][1], face_rect[i][1]+face_window,face_rect[i][0],face_rect[i][0]+face_window)
            
            image = image[face_rect[i][1]:face_rect[i][1]+face_window,face_rect[i][0]:face_rect[i][0]+face_window]
            images.append(image)
           
        img_concat = np.concatenate((images[0], images[1],images[2],images[3]), axis = 1)
        
        faces = face_cascade.detectMultiScale(images[2], 1.2,4,minSize=(60, 60), maxSize=(100, 100))
        print(type(faces), len(faces))
        for (x,y,w,h) in faces:
            print(x,y,x+w,y+h,w,h )
            cv2.rectangle(img_concat,(x,y),(x+w,y+h),(255,0,0),2)
            
        # save face rectangle in file
        
        if show_img == True:
            cv2.imshow(pattern,img_concat)
            cv2.waitKey(0)
            cv2.destroyAllWindows()  
            
        face_rect = [[120, 35],[120, 35], [80,35], [90, 30]]

In [57]:
data = []

In [59]:
tic = time.clock()
for c in classes:
    for p in persons:
        for i in range (1,10):
            pattern = prefix + p + wildcard + str(c) + suffix.format(i)
            files_list = glob.glob(pattern)
            #print(pattern, len(files_list), files_list)
            
            if len(files_list) > 0:
                if get_face_rect == True:
                    get_face_rectangle_viola_jones(c, p, data, files_list)
                else:
                    get_face_rect_default(c, p, data, files_list)     
print(time.clock() - tic)

8.356682835251888


In [60]:
print(len(data))

1235


In [61]:
# save file
df = pd.DataFrame(data, columns = ['class','person', 
                                   'depfile_path','depxs', 'depys', 'depxe','depye',
                                   'ir1file_path','ir1xs', 'ir1ys', 'ir1xe','ir1ye',
                                   'ir2file_path','ir2xs', 'ir2ys', 'ir2xe','ir2ye',
                                   'rgbfile_path','rgbxs', 'rgbys', 'rgbxe','rgbye'])
filename = 'training.csv'
df.to_csv(filename, index=False, encoding='utf-8')

In [62]:
df.head()

,class,person,depfile_path,depxs,depys,depxe,depye,ir1file_path,ir1xs,ir1ys,...,ir2file_path,ir2xs,ir2ys,ir2xe,ir2ye,rgbfile_path,rgbxs,rgbys,rgbxe,rgbye
0,1,p002,.\p002\02_1_depth_1.bmp,35,120,215,300,.\p002\02_1_ir1_1.bmp,35,120,...,.\p002\02_1_ir2_1.bmp,35,80,215,260,.\p002\02_1_rgb_1.bmp,30,90,210,270
1,1,p002,.\p002\02_1_depth_2.bmp,35,120,215,300,.\p002\02_1_ir1_2.bmp,35,120,...,.\p002\02_1_ir2_2.bmp,35,80,215,260,.\p002\02_1_rgb_2.bmp,30,90,210,270
2,1,p002,.\p002\02_1_depth_3.bmp,35,120,215,300,.\p002\02_1_ir1_3.bmp,35,120,...,.\p002\02_1_ir2_3.bmp,35,80,215,260,.\p002\02_1_rgb_3.bmp,30,90,210,270
3,1,p002,.\p002\02_1_depth_4.bmp,35,120,215,300,.\p002\02_1_ir1_4.bmp,35,120,...,.\p002\02_1_ir2_4.bmp,35,80,215,260,.\p002\02_1_rgb_4.bmp,30,90,210,270
4,1,p002,.\p002\02_1_depth_5.bmp,35,120,215,300,.\p002\02_1_ir1_5.bmp,35,120,...,.\p002\02_1_ir2_5.bmp,35,80,215,260,.\p002\02_1_rgb_5.bmp,30,90,210,270


In [67]:
df.shape

(1235, 22)